## Union

In [3]:
import re
import difflib
import pickle
from itertools import product
import sys
import numpy as np
import pandas as pd
import torch
from scipy.stats import hmean
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModel


### Config

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---- model & data paths
xgb_model = pickle.load(open('XGboost.pkl', 'rb'))
PATH1 = "../Datasets/Union +/Test 1/coin_Aave.csv"           # <-- change if needed
PATH2 = "../Datasets/Union +/Test 1/coin_BinanceCoin copy.csv"    # <-- change if needed
OUTPUT_PATH = "merged_union.csv"                                                      # <-- output path

# ---- validator fusion knobs
DECISION_THRESHOLD = 0.72   # final cutoff on fused score (0..1)
NAME_WEIGHT       = 0.85    # trust naming more (0..1)
MODEL_WEIGHT      = 0.15    # trust model prob (0..1)
COMPATIBILITY_CUTOFF = 0.70 # coverage-based score to allow union

# ---- MERGE CONFIG (union behavior)
ADD_SOURCE_COL      = True          # add __source column indicating origin dataset
SOURCE_LABEL_A      = "A"           # label for PATH1 rows
SOURCE_LABEL_B      = "B"           # label for PATH2 rows
FAIL_IF_INCOMPATIBLE= False         # if True: raise when compatibility < cutoff
COERCE_NUMERIC_MIN_VALID_FRAC = 0.70   # try numeric coercion if >=70% values convert
COERCE_DATETIME_MIN_VALID_FRAC = 0.70  # try datetime coercion if >=70% values convert

Using device: cuda


### Helpers

In [5]:
def preprocess_keyword(keyword):
    return keyword.replace("_", " ").lower()

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert").to(device)

def generate_embeddings(keywords, tokenizer, model, device):
    inputs = tokenizer(keywords, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embeddings

def normalize_name(s: str) -> str:
    """
    Aggressive normalization used for union-safe name matching:
    - lowercases
    - converts separators (_ - . / \ and spaces) to single spaces
    - removes non-alphanumeric chars (except space)
    - collapses spaces
    """
    s = s.lower()
    s = re.sub(r"[_\-\.\s/\\]+", " ", s)
    s = re.sub(r"[^a-z0-9 ]+", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def token_set(s: str):
    return set(normalize_name(s).split())

def is_pure_substring(a_raw: str, b_raw: str) -> bool:
    """
    True if one normalized string is a strict substring of the other (and not equal).
    We reject such 'matches' so GDP != GDP per capita.
    """
    a = normalize_name(a_raw).replace(" ", "")
    b = normalize_name(b_raw).replace(" ", "")
    return (a != b) and (a in b or b in a)

def name_similarity(a_raw: str, b_raw: str) -> float:
    """
    Union-safe similarity:
      - exact normalized equality → 1.0
      - penalize pure substring relations → 0.0
      - otherwise blend token Jaccard + character ratio + light length sanity
    """
    a_norm = normalize_name(a_raw)
    b_norm = normalize_name(b_raw)

    if a_norm == b_norm:
        return 1.0

    if is_pure_substring(a_raw, b_raw):
        return 0.0

    char_ratio = difflib.SequenceMatcher(
        None, a_norm.replace(" ", ""), b_norm.replace(" ", "")
    ).ratio()

    A, B = token_set(a_raw), token_set(b_raw)
    jacc = len(A & B) / len(A | B) if (A or B) else 0.0

    score = 0.6 * jacc + 0.4 * char_ratio

    len_ratio = (
        min(len(a_norm), len(b_norm)) / max(len(a_norm), len(b_norm))
        if max(len(a_norm), len(b_norm)) > 0 else 0
    )
    score = 0.85 * score + 0.15 * len_ratio
    return float(score)

def read_headers(csv_path: str):
    return list(pd.read_csv(csv_path, nrows=0).columns)

<>:18: SyntaxWarning: invalid escape sequence '\ '
<>:18: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_73606/3541259347.py:18: SyntaxWarning: invalid escape sequence '\ '
  - converts separators (_ - . / \ and spaces) to single spaces


### Read Schemas & Embeddings

In [6]:
cols1 = read_headers(PATH1)
cols2 = read_headers(PATH2)
print(f"Dataset1 columns: {len(cols1)} | Dataset2 columns: {len(cols2)}")

embeddings_1 = generate_embeddings([preprocess_keyword(c) for c in cols1], tokenizer, model, device)
embeddings_2 = generate_embeddings([preprocess_keyword(c) for c in cols2], tokenizer, model, device)

Dataset1 columns: 10 | Dataset2 columns: 9


### Build All Cross-Pairs

In [7]:
pair_embeddings = []
column_pairs    = []
for i, e1 in enumerate(embeddings_1):
    for j, e2 in enumerate(embeddings_2):
        pair_embeddings.append(np.concatenate([e1, e2]))
        column_pairs.append((cols1[i], cols2[j]))
pair_embeddings = np.vstack(pair_embeddings)

### Scale Features

In [8]:
scaler = StandardScaler().fit(pair_embeddings)
pair_scaled = scaler.transform(pair_embeddings)

### Scores

In [9]:
model_probs = xgb_model.predict_proba(pair_scaled)[:, 1]
name_scores = np.array([name_similarity(a, b) for (a, b) in column_pairs])

w_name, w_model = NAME_WEIGHT, MODEL_WEIGHT
fused_scores    = w_name * name_scores + w_model * model_probs

preds_fused     = (fused_scores >= DECISION_THRESHOLD).astype(int)

### Accepted Pairs & COVERAGE

In [10]:
compatible_pairs = [
    (a, b, s, mp, ns)
    for (a, b), s, mp, ns, p in zip(column_pairs, fused_scores, model_probs, name_scores, preds_fused)
    if p == 1
]

matched_A = {a for a, _, _, _, _ in compatible_pairs}
matched_B = {b for _, b, _, _, _ in compatible_pairs}

coverage_A = len(matched_A) / len(cols1) if cols1 else 0.0
coverage_B = len(matched_B) / len(cols2) if cols2 else 0.0
compatibility_score = (hmean([coverage_A, coverage_B]) if coverage_A and coverage_B else 0.0)

print("\n🔗 Compatible Column Pairs (accepted by manual fusion):")
if compatible_pairs:
    for a, b, s, mp, ns in sorted(compatible_pairs, key=lambda t: -t[2]):
        print(f"- {a} ↔ {b} | fused={s:.3f} (name={ns:.3f}, model={mp:.3f})")
else:
    print("No compatible columns found.")

is_compatible = (compatibility_score >= COMPATIBILITY_CUTOFF)
print("Datasets are COMPATIBLE (union-safe)" if is_compatible else "Datasets are NOT compatible (union-safe)")
print(f"Compatibility Score (coverage-based): {compatibility_score:.2f}")

# >>> HARD GUARD: abort before any union steps
if not is_compatible:
    print(f"\n⛔ Union aborted: compatibility {compatibility_score:.2f} is below cutoff {COMPATIBILITY_CUTOFF:.2f}.")
    sys.exit(2)   # non-zero exit so pipelines can detect the abort


🔗 Compatible Column Pairs (accepted by manual fusion):
- Open ↔ Open | fused=0.982 (name=1.000, model=0.881)
- Volume ↔ Volume | fused=0.975 (name=1.000, model=0.835)
- Symbol ↔ Symbol | fused=0.964 (name=1.000, model=0.763)
- Close ↔ Close | fused=0.956 (name=1.000, model=0.706)
- High ↔ High | fused=0.938 (name=1.000, model=0.588)
- SNo ↔ SNo | fused=0.896 (name=1.000, model=0.306)
- Low ↔ Low | fused=0.892 (name=1.000, model=0.283)
- Date ↔ Date | fused=0.885 (name=1.000, model=0.236)
- Name ↔ Name | fused=0.859 (name=1.000, model=0.060)
Datasets are COMPATIBLE (union-safe)
Compatibility Score (coverage-based): 0.95


### One-To-One Mapping

In [11]:
# We select a 1:1 mapping from Dataset2->Dataset1, highest fused first.
pairs_sorted = sorted(compatible_pairs, key=lambda t: -t[2])
A_used, B_used = set(), set()
mapping_B_to_A = {}   # {B_col: A_col}

for a, b, s, _, _ in pairs_sorted:
    if a not in A_used and b not in B_used:
        mapping_B_to_A[b] = a
        A_used.add(a)
        B_used.add(b)

print("\n✅ Selected 1:1 column mapping (Dataset2 → Dataset1):")
if mapping_B_to_A:
    for b, a in mapping_B_to_A.items():
        print(f"- {b}  →  {a}")
else:
    print("No 1:1 mappings selected (union will still proceed with unmatched columns).")



✅ Selected 1:1 column mapping (Dataset2 → Dataset1):
- Open  →  Open
- Volume  →  Volume
- Symbol  →  Symbol
- Close  →  Close
- High  →  High
- SNo  →  SNo
- Low  →  Low
- Date  →  Date
- Name  →  Name


### Read Full Data

In [12]:
df1 = pd.read_csv(PATH1, low_memory=False)
df2 = pd.read_csv(PATH2, low_memory=False)

# Provenance (optional)
if ADD_SOURCE_COL:
    df1["__source"] = SOURCE_LABEL_A
    df2["__source"] = SOURCE_LABEL_B


### Renaming Dataset 2 Columns Usings Selected Mapping

In [13]:
df2 = df2.rename(columns=mapping_B_to_A)

### D-Type Harmonization

In [14]:
def try_coerce_numeric(s: pd.Series) -> (pd.Series, float):
    coerced = pd.to_numeric(s, errors="coerce")
    valid_frac = 1.0 - (coerced.isna() & s.notna()).mean()
    return coerced, valid_frac

def try_coerce_datetime(s: pd.Series) -> (pd.Series, float):
    coerced = pd.to_datetime(s, errors="coerce", utc=False, infer_datetime_format=True)
    valid_frac = 1.0 - (coerced.isna() & s.notna()).mean()
    return coerced, valid_frac

def harmonize_pair(col: str, dfa: pd.DataFrame, dfb: pd.DataFrame):
    sa = dfa[col]
    sb = dfb[col]
    # Attempt numeric
    ca, fa = try_coerce_numeric(sa)
    cb, fb = try_coerce_numeric(sb)
    if fa >= COERCE_NUMERIC_MIN_VALID_FRAC and fb >= COERCE_NUMERIC_MIN_VALID_FRAC:
        dfa[col] = ca.astype("float64")
        dfb[col] = cb.astype("float64")
        return

    # Attempt datetime
    ca, fa = try_coerce_datetime(sa)
    cb, fb = try_coerce_datetime(sb)
    if fa >= COERCE_DATETIME_MIN_VALID_FRAC and fb >= COERCE_DATETIME_MIN_VALID_FRAC:
        dfa[col] = ca
        dfb[col] = cb
        return

    # Fallback: object (string)
    dfa[col] = dfa[col].astype("object")
    dfb[col] = dfb[col].astype("object")

# Harmonize dtypes on intersecting columns (after renaming)
common_cols = [c for c in df1.columns if c in df2.columns]
for c in common_cols:
    harmonize_pair(c, df1, df2)

/tmp/ipykernel_73606/3791693906.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  coerced = pd.to_datetime(s, errors="coerce", utc=False, infer_datetime_format=True)
/tmp/ipykernel_73606/3791693906.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  coerced = pd.to_datetime(s, errors="coerce", utc=False, infer_datetime_format=True)
/tmp/ipykernel_73606/3791693906.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004

### Unified Schema & Union

In [15]:
# Put Dataset1 columns first, then any extra columns from Dataset2
union_cols = list(df1.columns) + [c for c in df2.columns if c not in df1.columns]

df1u = df1.reindex(columns=union_cols)
df2u = df2.reindex(columns=union_cols)

df_merged = pd.concat([df1u, df2u], ignore_index=True)

### Report & Save

In [16]:
only_in_A = [c for c in df1.columns if c not in df2.columns]
only_in_B_after_rename = [c for c in df2.columns if c not in df1.columns]
both = [c for c in df1.columns if c in df2.columns]

print("\n📦 Union summary:")
print(f"- Rows: A={len(df1)}, B={len(df2)}, merged={len(df_merged)}")
print(f"- Columns in both: {len(both)}")
print(f"- Columns only in A: {len(only_in_A)}")
print(f"- Columns only in B (after rename): {len(only_in_B_after_rename)}")

df_merged.to_csv(OUTPUT_PATH, index=False)
print(f"\n💾 Saved merged dataset to: {OUTPUT_PATH}")


📦 Union summary:
- Rows: A=275, B=1442, merged=1717
- Columns in both: 10
- Columns only in A: 1
- Columns only in B (after rename): 0

💾 Saved merged dataset to: merged_union.csv
